In [1]:
try:
    # Comment out if not using colab
    from google.colab import drive
    drive.mount('/content/drive')

    # Specific for luca's computer
    %cd "/content/drive/Othercomputers/lucas-yoga/Current/INFO381/code/INFO381-GitHub"
    using_colab = True
except:
    print("Not using Google Colab")
    using_colab = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/lucas-yoga/Current/INFO381/code/INFO381-GitHub


In [2]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2v49vero
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-2v49vero
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import accuracy_score
import clip


from utils import get_dataloaders
from model_definitions import CLIPClassifier

In [14]:
# Define resize transform for the CNN
resize_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# Load both train and test dataloaders for the CNN model
cnn_train_loader, cnn_test_loader = get_dataloaders(
    zip_path="fake_vs_real.zip",
    batch_size=32,
    split='both',
    transform=resize_transform
)

# Setup the ResNet18 model
cnn_model = models.resnet18(pretrained=False)
num_ftrs = cnn_model.fc.in_features
cnn_model.fc = nn.Linear(num_ftrs, 2)
cnn_model.load_state_dict(torch.load("models/resnet18_cnn.pth"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model.to(device)
cnn_model.eval()

# Define evaluation function (same as for CLIP)
def evaluate(model, dataloader):
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Evaluate on train and test sets
accuracy_train_cnn = evaluate(cnn_model, cnn_train_loader)
accuracy_test_cnn = evaluate(cnn_model, cnn_test_loader)

print(f"ResNet18 Train Accuracy: {accuracy_train_cnn:.2f}")
print(f"ResNet18 Test Accuracy: {accuracy_test_cnn:.2f}")


Running in Google Colab


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


ResNet18 Train Accuracy: 0.79
ResNet18 Test Accuracy: 0.79


In [13]:
# 1) Load the CLIP backbone and our classifier model
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Load both train and test dataloaders
clip_train_loader, clip_test_loader = get_dataloaders(zip_path="fake_vs_real.zip", batch_size=32, split='both', transform=preprocess)

model = CLIPClassifier(clip_model, embed_dim=512, num_classes=2).to(device)
model.load_state_dict(torch.load("models/clip_classifier_10epochs.pth", map_location=device))
model.eval()

# 2) Define an evaluation function
from sklearn.metrics import accuracy_score

def evaluate(model, dataloader):
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# 3) Evaluate on both sets
accuracy_train = evaluate(model, clip_train_loader)
accuracy_test = evaluate(model, clip_test_loader)

print(f"CLIP ViT Train Accuracy: {accuracy_train:.2f}")
print(f"CLIP ViT Test Accuracy: {accuracy_test:.2f}")


Running in Google Colab


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


CLIP ViT Train Accuracy: 0.97
CLIP ViT Test Accuracy: 0.95
